CHATBOT FTMM

In [1]:
import json
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.memory import ConversationBufferWindowMemory
import os
from tqdm import tqdm
import chromadb

In [2]:
loader = DirectoryLoader("knowledges")
docs = loader.load()

In [3]:
client = chromadb.PersistentClient(path="database")
collection_ftmm = client.get_or_create_collection(name="ftmm")

In [4]:
client.reset()

True

In [10]:
metadata = [docs[i].metadata for i in range(len(docs))]
documents = [docs[i].page_content for i in range(len(docs))]
ids = [f"doc{i}" for i in range(len(docs))]

In [8]:
embed = OpenAIEmbeddings()
embeddings = embed.embed_documents(documents)

In [40]:
status = collection_ftmm.add(
    ids=ids,
    documents=documents,
    embeddings=embeddings,
    metadatas=metadata
)

Add of existing embedding ID: doc0
Add of existing embedding ID: doc1
Add of existing embedding ID: doc2
Add of existing embedding ID: doc3
Add of existing embedding ID: doc4
Add of existing embedding ID: doc5
Add of existing embedding ID: doc6
Insert of existing embedding ID: doc0
Insert of existing embedding ID: doc1
Insert of existing embedding ID: doc2
Insert of existing embedding ID: doc3
Insert of existing embedding ID: doc4
Insert of existing embedding ID: doc5
Insert of existing embedding ID: doc6


In [35]:
docsearch = Chroma(persist_directory="database", embedding_function=embed, collection_name="ftmm")
docsearch._collection.count()

7

In [135]:
# LLM
chat_model = ChatOpenAI()

# Template prompt
template = """Jawablah pertanyaan di bawah ini berdasarkan konteks yang diberikan! \
Jika dalam pertanyaan merujuk ke histori chat sebelumnya, maka gunakan konteks dari pertanyaan \
sebelumnya untuk menjawab!
Konteks:
{context}

Pertanyaan:
{question}
"""

template_system = """Namamu adalah Iris, sebuah chatbot Fakultas Teknologi Maju dan \
Multidisiplin (FTMM), Universitas Airlangga. Kamu siap menjawab pertanyaan apapun \
seputar FTMM. Kamu menjawab setiap pertanyaan dengan ceria, sopan, dan asik!
"""

# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(template_system),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template(template),
    ]
)

# Memory
memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True, k=1)

In [136]:
chain = prompt | chat_model

In [143]:
question = "ceritakan sejarah FTMM!"
contexts = docsearch.similarity_search(question, k=3)
response = chain.invoke({"context": contexts, "question": question, "chat_history": memory.buffer_as_messages})

# Add response to memory
memory.save_context({"input": template.format(context=contexts, question=question)}, 
                    {"output": response.content})
response

AIMessage(content='Fakultas Teknologi Maju dan Multidisiplin (FTMM) didirikan pada bulan November 2019 dengan nama awal "Sekolah Teknologi Maju dan Multidisiplin". FTMM hadir sebagai pelengkap dari tiga pilar utama Universitas Airlangga yaitu ilmu sosial, ilmu kehidupan, dan ilmu kesehatan, dengan menambahkan pilar baru di bidang keteknikan.\n\nFTMM memiliki lima program studi S1 yang terdiri dari Teknik Robotika dan Kecerdasan Buatan, Teknologi Sains Data, Rekayasa Nanoteknologi, Teknik Industri, dan Teknik Elektro. Pada tahun pertama, jumlah mahasiswa FTMM sekitar 400 orang.\n\nSebagai fakultas baru, FT')

In [147]:
import tiktoken
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [148]:
folder = "Website FTMM"
for i in range(len(os.listdir(folder))):
    file_path = os.path.join(folder, os.listdir(folder)[i])
    with open(file_path, "r") as f:
        content = f.read()
    
    tokens = encoder.encode(content)
    print(len(tokens))

    break

1177


In [151]:
contexts = docsearch.similarity_search("apa itu arta", k=3)
temp = f"{contexts}"

tokens = encoder.encode(temp)
len(tokens)

2081

In [153]:
tokens = encoder.encode(f"{memory.buffer_as_messages}")
len(tokens)

1780

In [154]:
contexts = docsearch.similarity_search("jadi apa itu arta?", k=3)
temp = f"{contexts}"

tokens = encoder.encode(temp)
len(tokens)

2081

In [168]:
saved_contexts = list()

contexts = [c for c in docsearch.similarity_search("apa itu arta?", k=3) if c not in saved_contexts]
print(len(contexts), len(saved_contexts))
saved_contexts = contexts

3 0


In [169]:
contexts = [c for c in docsearch.similarity_search("berarti apa tujuan arta?", k=3) if c not in saved_contexts]
print(len(contexts), len(saved_contexts))
# saved_contexts = contexts

1 3


In [171]:
saved_contexts.extend(contexts)

In [173]:
tokens = encoder.encode(f"{saved_contexts}")
len(tokens)

2323

In [175]:
contexts = docsearch.similarity_search("berarti apa tujuan arta?", k=3)

In [178]:
temp = prompt.format(context=contexts, question="berarti apa tujuan arta?", chat_history=memory.buffer_as_messages)
tokens = encoder.encode(temp)
len(tokens)

3993